In [7]:
from gpytorch.distributions import MultivariateNormal
import torch

# Define mean and covariance
mean = torch.zeros(3).reshape(-1, 1)
covariance = torch.tensor([[1.0, 0.5, 0.2],
                           [0.5, 1.0, 0.3],
                           [0.2, 0.3, 1.0]])

# Create MultivariateNormal
mvn = MultivariateNormal(mean, covariance)
mvn

MultivariateNormal(loc: torch.Size([3, 1]), covariance_matrix: torch.Size([3, 3, 3]))

In [9]:
mvn.rsample()

RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 2.  Target sizes: [3, 1, 1].  Tensor sizes: [3, 3]

In [15]:
import torch
import torch.nn as nn
import numpy as np

class MockDistribution:
    def __init__(self, mean, cov):
        self.mean = mean  # Mean of the distribution
        self.cov = cov    # Covariance of the distribution
    
    def sample(self):
        # Simulate sampling from the distribution
        return torch.randn_like(self.mean) * torch.sqrt(self.cov) + self.mean

class RegressionExample(nn.Module):
    def __init__(self, in_features, out_features):
        super(RegressionExample, self).__init__()
        # Define mock posterior for weights
        self.weight_mean = torch.randn(out_features, in_features)
        self.weight_cov = torch.abs(torch.randn(out_features, in_features))  # Variance must be non-negative
        self.W_dist = MockDistribution(self.weight_mean, self.weight_cov)
        
        # Define mock posterior for noise
        self.noise_mean = torch.zeros(out_features)
        self.noise_logdiag = torch.randn(out_features) * np.log(0.1)
        self.noise_scale = torch.exp(self.noise_logdiag)
        self.noise_dist = MockDistribution(self.noise_mean, self.noise_scale)

    def W(self):
        # Sample weights from the posterior
        return self.W_dist.sample()

    def noise(self):
        # Sample noise from the posterior
        return self.noise_dist.sample()

    def predictive(self, x):
        # Compute predictive distribution
        return (self.W() @ x[..., None]).squeeze(-1) + self.noise()

# Example usage
torch.manual_seed(0)  # For reproducibility

# Initialize model
model = RegressionExample(in_features=3, out_features=1)

# Create some random input data
x = torch.randn(4, 3)  # Batch of 4, 3 input features

# Get predictive distribution
output = model.predictive(x)
print("Input:\n", x)
print("Output (predictive distribution):\n", output)


Input:
 tensor([[ 0.8380, -0.7193, -0.4033],
        [-0.5966,  0.1820, -0.8567],
        [ 1.1006, -1.0712,  0.1227],
        [-0.5663,  0.3731, -0.8920]])
Output (predictive distribution):
 tensor([[1.0463],
        [0.7578],
        [0.8796],
        [0.8038]])


In [10]:
covariance.squeeze(1)

tensor([[1.0000, 0.5000, 0.2000],
        [0.5000, 1.0000, 0.3000],
        [0.2000, 0.3000, 1.0000]])

In [12]:
covariance.squeeze(0)

tensor([[1.0000, 0.5000, 0.2000],
        [0.5000, 1.0000, 0.3000],
        [0.2000, 0.3000, 1.0000]])

In [13]:
import torch

# Define dimensions
batch_size = 4
in_features = 3
out_features = 2

# Random weights matrix (out_features x in_features)
W = torch.tensor([[0.5, -0.2, 0.1],
                  [0.4, 0.7, -0.3]])

# Random input matrix (batch_size x in_features)
x = torch.tensor([[1.0, 2.0, 3.0],
                  [4.0, 5.0, 6.0],
                  [7.0, 8.0, 9.0],
                  [10.0, 11.0, 12.0]])

# Random noise vector (out_features)
noise = torch.tensor([0.1, -0.2])

# Step-by-step computation
# 1. Reshape x to (batch_size, in_features, 1)
x_column = x[..., None]  # Shape: (4, 3, 1)

# 2. Matrix multiplication
Wx = torch.matmul(W, x_column)  # Shape: (4, 2, 1)

# 3. Squeeze the last dimension
Wx_squeezed = Wx.squeeze(-1)  # Shape: (4, 2)

# 4. Add noise
output = Wx_squeezed + noise  # Shape: (4, 2)

# Print results
print("Input matrix (x):\n", x)
print("Weights matrix (W):\n", W)
print("Noise vector:\n", noise)
print("Output:\n", output)


Input matrix (x):
 tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])
Weights matrix (W):
 tensor([[ 0.5000, -0.2000,  0.1000],
        [ 0.4000,  0.7000, -0.3000]])
Noise vector:
 tensor([ 0.1000, -0.2000])
Output:
 tensor([[0.5000, 0.7000],
        [1.7000, 3.1000],
        [2.9000, 5.5000],
        [4.1000, 7.9000]])
